In [16]:
# upload audio file
from google.colab import files
uploaded = files.upload()
path = next(iter(uploaded))

Saving kovoiceinhindi (1).wav to kovoiceinhindi (1).wav


In [17]:
num_speakers = 2 #@param {type:"integer"}

language = 'any' #@param ['any', 'English']

model_size = 'large' #@param ['tiny', 'base', 'small', 'medium', 'large']


model_name = model_size
if language == 'English' and model_size != 'large':
  model_name += '.en'


In [18]:
!pip install -q git+https://github.com/openai/whisper.git > /dev/null
!pip install -q git+https://github.com/pyannote/pyannote-audio > /dev/null

import whisper
import datetime

import subprocess

import torch
import pyannote.audio
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding
embedding_model = PretrainedSpeakerEmbedding(
    "speechbrain/spkrec-ecapa-voxceleb",
    device=torch.device("cuda"))

from pyannote.audio import Audio
from pyannote.core import Segment

import wave
import contextlib

from sklearn.cluster import AgglomerativeClustering
import numpy as np

In [19]:
if path[-3:] != 'wav':
  subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
  path = 'audio.wav'

In [ ]:
model = whisper.load_model(model_size)

In [ ]:
result = model.transcribe(path)
segments = result["segments"]

In [7]:
print(segments)

[{'id': 0, 'seek': 0, 'start': 0.0, 'end': 1.0, 'text': ' 안녕하세요', 'tokens': [50365, 19289, 50415], 'temperature': 0.0, 'avg_logprob': -0.32103013092616817, 'compression_ratio': 1.3823529411764706, 'no_speech_prob': 0.15236800909042358}, {'id': 1, 'seek': 0, 'start': 1.0, 'end': 2.0, 'text': ' 그의 전화', 'tokens': [50415, 4296, 2785, 6831, 11604, 50465], 'temperature': 0.0, 'avg_logprob': -0.32103013092616817, 'compression_ratio': 1.3823529411764706, 'no_speech_prob': 0.15236800909042358}, {'id': 2, 'seek': 0, 'start': 2.0, 'end': 3.0, 'text': ' 안녕 여러분', 'tokens': [50465, 13810, 14707, 50515], 'temperature': 0.0, 'avg_logprob': -0.32103013092616817, 'compression_ratio': 1.3823529411764706, 'no_speech_prob': 0.15236800909042358}, {'id': 3, 'seek': 0, 'start': 3.0, 'end': 6.0, 'text': ' 오늘은 24시간 동안 한국말 쓰기', 'tokens': [50515, 23720, 4022, 31099, 32589, 21045, 11023, 17373, 2401, 50665], 'temperature': 0.0, 'avg_logprob': -0.32103013092616817, 'compression_ratio': 1.3823529411764706, 'no_speec

In [8]:
with contextlib.closing(wave.open(path,'r')) as f:
  frames = f.getnframes()
  rate = f.getframerate()
  duration = frames / float(rate)

In [11]:
audio = Audio()

def segment_embedding(segment):
  start = segment["start"]
  # Whisper overshoots the end timestamp in the last segment
  end = min(duration, segment["end"])
  clip = Segment(start, end)
  waveform, sample_rate = audio.crop(path, clip)
  # Ensure single channel (mono)
  if waveform.shape[0] > 1:
      waveform = waveform.mean(axis=0, keepdims=True)
  return embedding_model(waveform[None])

In [12]:
embeddings = np.zeros(shape=(len(segments), 192))
for i, segment in enumerate(segments):
  embeddings[i] = segment_embedding(segment)

embeddings = np.nan_to_num(embeddings)

In [13]:
clustering = AgglomerativeClustering(num_speakers).fit(embeddings)
labels = clustering.labels_
for i in range(len(segments)):
  segments[i]["speaker"] = 'SPEAKER ' + str(labels[i] + 1)

In [14]:
def time(secs):
  return datetime.timedelta(seconds=round(secs))

f = open("transcript.txt", "w")

for (i, segment) in enumerate(segments):
  if i == 0 or segments[i - 1]["speaker"] != segment["speaker"]:
    f.write("\n" + segment["speaker"] + ' ' + str(time(segment["start"])) + '\n')
  f.write(segment["text"][1:] + ' ')
f.close()